In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
import os
from bs4 import BeautifulSoup

os.environ["PATH"] += os.pathsep + ""
options = Options()
options.headless = True  # Running browser in headless mode
geckodriver_path = "geckodriver.exe"
driver = webdriver.Firefox(options=options)
url = "https://app.alt.xyz/browse/liquid-auctions?sortBy=highest_alt_value_first "
driver.get(url)
content = driver.page_source
driver.quit()
soup = BeautifulSoup(content, 'html5lib')
base_url = 'https://app.alt.xyz'
links = [a["href"] for a in soup.find_all('a', class_="css-dcq7s5")]

cards_n_player_number = [span.text for span in soup.find_all('span', class_="MuiTypography-root MuiTypography-vegaBody2 MuiTypography-noWrap css-1iu657u")]
card = cards_n_player_number[::2]
player_n_number = cards_n_player_number[1::2]

grade = [span.text for span in soup.find_all('span', class_="MuiTypography-root MuiTypography-vegaBody2 css-1e0sb4z")]
grade = [item for item in grade if item != 'No options.']

prices = [span.text for span in soup.find_all('span', class_="MuiTypography-root MuiTypography-vegaH7 css-yrl21b")]
auction_data = []
for i in range(len(prices)):
    auction_dict = {}
    auction_dict['link'] = base_url + links[i]
    auction_dict['card'] = card[i]
    auction_dict['player'] = player_n_number[i].split('#')[0]
    if len(player_n_number[i].split('#')) > 1:
        auction_dict['card_number'] = player_n_number[i].split('#')[1]
    auction_dict['grade'] = grade[i]
    auction_data.append(auction_dict)
df_auction = pd.DataFrame(auction_data)
df_auction


,link,card,player,grade,card_number
0,https://app.alt.xyz/browse/external-listing?id...,1998 Pokemon Promo Japanese,Illustrator Pikachu,PSA 9,NaN
1,https://app.alt.xyz/browse/external-listing?id...,1909 T206 Sweet Caporal,Ty Cobb/Portrait Green,PSA 8,97
2,https://app.alt.xyz/browse/external-listing?id...,1909 T206 Sweet Caporal,Ty Cobb/Portrait Green,PSA 8,97
3,https://app.alt.xyz/browse/external-listing?id...,2000 Playoff Contenders Championship Ticket Au...,Tom Brady,BGS 8 Auto 10,144
4,https://app.alt.xyz/browse/external-listing?id...,2000 Playoff Contenders Championship Ticket Au...,Tom Brady,BGS 8 Auto 10,144
5,https://app.alt.xyz/browse/external-listing?id...,2020 National Treasures Rookie Patch Autograph,Justin Herbert,PSA 10 Auto 10,158
6,https://app.alt.xyz/browse/external-listing?id...,2020 National Treasures Rookie Patch Autograph,Justin Herbert,PSA 10 Auto 10,158
7,https://app.alt.xyz/browse/external-listing?id...,1965 Topps,Joe Namath,PSA 8.5,122
8,https://app.alt.xyz/browse/external-listing?id...,1965 Topps,Joe Namath,PSA 8.5,122
9,https://app.alt.xyz/browse/external-listing?id...,2009 Topps Chrome Gold Refractor,Stephen Curry,BGS 9.5,101


In [ ]:
options = webdriver.FirefoxOptions()
options.headless = True

stats_list = []

for url in df_auction['link']:
    with webdriver.Firefox(options=options) as driver:
        try:
            driver.get(url)

            # Get the default div's content
            default_div = driver.find_element(By.CSS_SELECTOR, '.MuiGrid-grid-sm-9')
            default_content = default_div.get_attribute('outerHTML')

            # Click the "View All" button to load the sidebar
            view_all_button = driver.find_element(By.CSS_SELECTOR, 'button.MuiButtonBase-root:nth-child(3)')
            view_all_button.click()

            # Wait for the sidebar's specific div to be visible
            wait = WebDriverWait(driver, 2)  # Wait for up to 2 seconds
            sidebar_div = wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, ".css-nltf5e > div:nth-child(3)")))

            # Get the HTML content of the div
            sidebar_content = sidebar_div.get_attribute('outerHTML')

            # Create Beautiful Soup objects from the contents
            soup_default = BeautifulSoup(default_content, 'html5lib')
            soup_sidebar = BeautifulSoup(sidebar_content, 'html5lib')

            # Logic to process the soups
            divs = soup_sidebar.find_all('div', class_="css-vxna0y")

            best_offer_text = [span.text for span in soup_sidebar.find_all('span', class_="MuiTypography-root MuiTypography-vegaSubtitle2 css-1gcetpq")]
            offer_price = [span.text.split('$')[1].replace(',','') for span in soup_sidebar.find_all('span', class_="MuiTypography-root MuiTypography-vegaSubtitle2 css-16tlq5a")]
            img_source = [div.find('img')['alt'] for div in divs]
            dates = [span.text for span in soup_sidebar.find_all('span', class_="MuiTypography-root MuiTypography-vegaBody3 css-11ntl8z")]
            offers_data = []
            for i in range(len(offer_price)):
                offers_dict = {}
                offers_dict['best_offer_text'] = best_offer_text[i]
                offers_dict['offer_price'] = offer_price[i]
                offers_dict['img_source'] = img_source[i]
                offers_dict['dates'] = dates[i]
                offers_data.append(offers_dict)

            df_price = pd.DataFrame(offers_data)

            df_price['offer_price'] = df_price['offer_price'].astype('int')
            df_price['dates'] = pd.to_datetime(df_price['dates'])
            
            # Only include sales within the past year. 
            df_price = df_price[df_price['dates'].dt.year==2022]
            
            df_stats = {}

            df_stats['min_price'] = 0.0
            df_stats['max_price'] = 0.0
            df_stats['mean_price'] = 0.0
            df_stats['median_price'] = 0.0

            # If there are more than 3 sales on eBay in the past year, use ONLY the ebay sales
            if len(df_price[df_price['img_source']=='eBay']) > 3:
                df_price = df_price[df_price['img_source']=='eBay']
                df_stats['min_price'] = df_price['offer_price'].min()
                df_stats['max_price'] = df_price['offer_price'].max()
                df_stats['mean_price'] = df_price['offer_price'].mean()
                df_stats['median_price'] = df_price['offer_price'].median()
            else:
                df_stats['min_price'] = df_price['offer_price'].min()
                df_stats['max_price'] = df_price['offer_price'].max()
                df_stats['mean_price'] = df_price['offer_price'].mean()
                df_stats['median_price'] = df_price['offer_price'].median()


            df_stats = pd.DataFrame(df_stats,index=[0])

            df_stats['breakeven_price'] = df_stats['mean_price'].apply(lambda x: x*0.62 if x < 2500 else 1560+(x-2500)*0.7)
            df_stats['make_10%_price'] = df_stats['mean_price'].apply(lambda x: x*0.56 if x < 2500 else 1400+(x-2500)*0.63)
            df_stats['dollar_under'] = df_stats['breakeven_price'] - (15600 / 1.2)
            df_stats['percent_under'] = 15600 / (1.2 * df_stats['breakeven_price'])
            df_stats['url'] = url
            
            stats_dict = dict(df_stats)
            
            stats_dict['min_fixed_price'] = min(int(num) for num in [span.text.split('$')[1].replace(',','') for span in soup_default.find_all('span', class_="MuiTypography-root MuiTypography-vegaSubtitle2 css-w9sxkt")])
            
            stats_list.append(stats_dict)

        except Exception as e:
            pass

df_stats = pd.DataFrame(stats_list)
df_auction = df_auction.merge(df_stats,left_on='link',right_on=['url'])
df_auction.drop(columns=['url'],inplace=True)
df_auction.to_csv('scraped_results_complete.csv',index=False)